In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from docx import Document
from docx.shared import Inches


#=============================
# 1.  LOAD DATA 
#=============================

df1 = pd.read_excel("for anomalies WORKED.xlsx", header=[0,1])


#=====================================
# 2. columns sortting and aggregation 
#=====================================

df1.columns = [f"{a}_{b}" if a not in ["date","year","month","day"] else a
              for a,b in df1.columns]

# Rename year/month
df1 = df1.rename(columns={"year":"Year","month":"Month"})
df1["Year"] = pd.to_numeric(df1["Year"], errors="coerce")
df1["Month"] = pd.to_numeric(df1["Month"], errors="coerce")

# Define seasons
season_map = {
    12: "DJF", 1: "DJF", 2: "DJF",
    3: "MAM", 4: "MAM", 5: "MAM",
    6: "JJA", 7: "JJA", 8: "JJA",
    9: "SON", 10: "SON", 11: "SON"
}
df1["Season"] = df1["Month"].map(season_map)


# --- FUNCTION TO GET MEAN OF PREFIX VARIABLES ---
def average_by_prefix(df1, prefix, var):
    cols = [c for c in df1.columns if c.startswith(prefix + "_" + var)]
    if cols:
        return df1[cols].mean(axis=1)
    return None

    
# ===============================
# 3. Regional mean and Substting
# ===============================

# Compute average SOUTH and NORTH variables (row-wise mean across suffixes)
df1["SOUTH_PPT_avg"] = average_by_prefix(df1, "SOUTH", "PPT")
df1["SOUTH_TMAX_avg"] = average_by_prefix(df1, "SOUTH", "TMAX")
df1["SOUTH_TMIN_avg"] = average_by_prefix(df1, "SOUTH", "TMIN")

df1["NORTH_PPT_avg"] = average_by_prefix(df1, "NORTH", "PPT")
df1["NORTH_TMAX_avg"] = average_by_prefix(df1, "NORTH", "TMAX")
df1["NORTH_TMIN_avg"] = average_by_prefix(df1, "NORTH", "TMIN")

# Nigeria average = mean of South and North
df1["NIGERIA_PPT"] = (df1["SOUTH_PPT_avg"] + df1["NORTH_PPT_avg"]) / 2
df1["NIGERIA_TMAX"] = (df1["SOUTH_TMAX_avg"] + df1["NORTH_TMAX_avg"]) / 2
df1["NIGERIA_TMIN"] = (df1["SOUTH_TMIN_avg"] + df1["NORTH_TMIN_avg"]) / 2




# Select ONLY the average columns
avg_df = df1[[
    "Year","Month","day","Season",
    "SOUTH_PPT_avg","SOUTH_TMAX_avg","SOUTH_TMIN_avg",
    "NORTH_PPT_avg","NORTH_TMAX_avg","NORTH_TMIN_avg",
    "NIGERIA_PPT","NIGERIA_TMAX","NIGERIA_TMIN"
]]

# Subset to 1970–2023
avg_df = avg_df[(avg_df["Year"]>=1970) & (avg_df["Year"]<=2023)]





# --- YEARLY AGGREGATES ---
#yearly = avg_df.groupby("Year").agg({
#    "SOUTH_PPT_avg": "sum",
#    "NORTH_PPT_avg": "sum",
#    "SOUTH_TMAX_avg": "mean",
#    "NORTH_TMAX_avg": "mean",
#    "SOUTH_TMIN_avg": "mean",
#    "NORTH_TMIN_avg": "mean",
#    "NIGERIA_PPT": "sum",
#    "NIGERIA_TMAX": "mean",
#    "NIGERIA_TMIN": "mean"
#}).reset_index()

# --- SEASONAL AGGREGATES ---
seasonal = avg_df.groupby(["Year", "Season"]).agg({
    "SOUTH_PPT_avg": "sum",
    "NORTH_PPT_avg": "sum",
    "SOUTH_TMAX_avg": "mean",
    "NORTH_TMAX_avg": "mean",
    "SOUTH_TMIN_avg": "mean",
    "NORTH_TMIN_avg": "mean",
    "NIGERIA_PPT": "sum",
    "NIGERIA_TMAX": "mean",
    "NIGERIA_TMIN": "mean"
})




# ===============================
# 4. Yearly Means
# ===============================
yearly = avg_df.groupby("Year")[["SOUTH_PPT_avg","SOUTH_TMAX_avg","SOUTH_TMIN_avg",
                                 "NORTH_PPT_avg","NORTH_TMAX_avg","NORTH_TMIN_avg",
                                 "NIGERIA_PPT","NIGERIA_TMAX","NIGERIA_TMIN"]].mean()

# ===============================
# 5. Baseline (1970–1999)
# ===============================
baseline = yearly.loc[1970:1999].mean()

# ===============================
# 6. Yearly Anomalies
# ===============================
yearly_anomalies = yearly - baseline
# ===============================
# 7. Seasonal Anomalies
# ===============================
#seasonal = avg_df.groupby(["Year","Season"])[["SOUTH_PPT_avg","NORTH_PPT_avg","NIGERIA_PPT"]].mean()
seasonal_clim = seasonal.groupby("Season").mean()
seasonal_anomalies = seasonal - seasonal_clim
# ===============================
# 8. Save Excel Results
# ===============================
with pd.ExcelWriter("anomalies_results.xlsx") as writer:
    yearly.to_excel(writer, sheet_name="Yearly_Means")
    yearly_anomalies.to_excel(writer, sheet_name="Yearly_Anomalies")
    seasonal.to_excel(writer, sheet_name="Seasonal_Means")
    seasonal_anomalies.to_excel(writer, sheet_name="Seasonal_Anomalies")
# ===============================
# 9. Plotting
# ===============================
def plot_anomalies(df_anom, var, ylabel):
    plt.figure(figsize=(12,7), dpi=300)
    plt.axhline(0, color="black", linestyle="--", linewidth=1)

    # Mean ±1σ shading (Nigeria as national average)
    if f"NIGERIA_{var}" in df_anom.columns:
        avg_series = df_anom[f"NIGERIA_{var}"]
        mean_val = avg_series.mean()
        std_val = avg_series.std()
        plt.fill_between(df_anom.index, mean_val-std_val, mean_val+std_val, 
                         color="gray", alpha=0.15, label="±1 Std Dev (Nigeria)")

    # Lines
    if f"SOUTH_{var}_avg" in df_anom.columns:
        plt.plot(df_anom.index, df_anom[f"SOUTH_{var}_avg"], linestyle="--", marker="o", 
                 label="South", color="royalblue", alpha=0.8)

    if f"NORTH_{var}_avg" in df_anom.columns:
        plt.plot(df_anom.index, df_anom[f"NORTH_{var}_avg"], linestyle="--", marker="s", 
                 label="North", color="firebrick", alpha=0.8)

    if f"NIGERIA_{var}" in df_anom.columns:
        plt.plot(df_anom.index, df_anom[f"NIGERIA_{var}"], linestyle="-", marker="D", 
                 label="Nigeria", color="black", linewidth=2)

    # Labels & Style
    plt.title(f"Yearly {var.upper()} Anomalies (South, North, Nigeria)", fontsize=16, fontweight="bold")
    plt.xlabel("Year", fontsize=14, fontweight="bold")
    plt.ylabel(ylabel, fontsize=14, fontweight="bold")
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(fontsize=12, frameon=True, loc="best")
    plt.grid(True, linestyle=":", alpha=0.6)

    plt.tight_layout()
    plt.savefig(f"{var}_anomalies.png", dpi=300)
    plt.close()

plot_anomalies(yearly_anomalies, "PPT", "Precipitation Anomaly (mm)")
plot_anomalies(yearly_anomalies, "TMAX", "Tmax Anomaly (°C)")
plot_anomalies(yearly_anomalies, "TMIN", "Tmin Anomaly (°C)")

# Seasonal PPT anomaly plots
seasons = ["DJF","MAM","JJA","SON"]
fig, axes = plt.subplots(2,2, figsize=(16,12), dpi=300, sharex=True)
axes = axes.flatten()

for i, season in enumerate(seasons):
    subset = seasonal_anomalies.loc[pd.IndexSlice[:, season], :]
    years = subset.index.get_level_values("Year")

    axes[i].axhline(0, color="black", linestyle="--", linewidth=1)
    axes[i].plot(years, subset["SOUTH_PPT_avg"], linestyle="--", marker="o", 
                 label="South", color="royalblue", alpha=0.8)
    axes[i].plot(years, subset["NORTH_PPT_avg"], linestyle="--", marker="s", 
                 label="North", color="firebrick", alpha=0.8)
    axes[i].plot(years, subset["NIGERIA_PPT"], linestyle="--", marker="v", 
                 label="Nigeria", color="black", alpha=0.8)

    axes[i].set_title(f"{season} PPT Anomalies", fontsize=14, fontweight="bold")
    axes[i].set_ylabel("Anomaly (mm)", fontsize=12, fontweight="bold")
    axes[i].tick_params(axis="x", labelsize=10)
    axes[i].tick_params(axis="y", labelsize=10)
    axes[i].grid(True, linestyle=":", alpha=0.6)

    if i == 0:
        axes[i].legend(fontsize=10)

plt.suptitle("Seasonal Precipitation Anomalies (South vs North)", fontsize=18, fontweight="bold")
plt.tight_layout(rect=[0,0,1,0.96])
plt.savefig("seasonal_ppt_anomalies.png", dpi=300)
plt.close()

# ===============================
# 10. Word Report
# ===============================
doc = Document()

# Title
doc.add_heading("Climate Anomalies Report", level=0)

# Intro
doc.add_paragraph(
    "This report summarizes climate anomalies for South and North regions "
    "based on precipitation (PPT), maximum temperature (TMAX), and minimum temperature (TMIN). "
    "The analysis includes yearly means, yearly anomalies, and seasonal anomalies for precipitation. "
    "Plots are included for visual interpretation."
)

# Yearly Means Table
doc.add_heading("Yearly Means", level=1)
t = doc.add_table(rows=1, cols=len(yearly.columns)+1)
hdr_cells = t.rows[0].cells
hdr_cells[0].text = "Year"
for i, col in enumerate(yearly.columns):
    hdr_cells[i+1].text = col

for year, row in yearly.iterrows():
    row_cells = t.add_row().cells
    row_cells[0].text = str(year)
    for i, val in enumerate(row):
        row_cells[i+1].text = f"{val:.2f}"

# Yearly Anomalies Table
doc.add_heading("Yearly Anomalies", level=1)
t = doc.add_table(rows=1, cols=len(yearly_anomalies.columns)+1)
hdr_cells = t.rows[0].cells
hdr_cells[0].text = "Year"
for i, col in enumerate(yearly_anomalies.columns):
    hdr_cells[i+1].text = col

for year, row in yearly_anomalies.iterrows():
    row_cells = t.add_row().cells
    row_cells[0].text = str(year)
    for i, val in enumerate(row):
        row_cells[i+1].text = f"{val:.2f}"

# Seasonal Anomalies Table
doc.add_heading("Seasonal Precipitation Anomalies", level=1)
t = doc.add_table(rows=1, cols=3)
hdr_cells = t.rows[0].cells
hdr_cells[0].text = "Year"
hdr_cells[1].text = "Season"
hdr_cells[2].text = "South / North Anomalies"

for (year, season), row in seasonal_anomalies.iterrows():
    row_cells = t.add_row().cells
    row_cells[0].text = str(year)
    row_cells[1].text = season
    row_cells[2].text = f"South: {row['SOUTH_PPT_avg']:.2f}, North: {row['NORTH_PPT_avg']:.2f}"

# Plots
doc.add_heading("Plots", level=1)

doc.add_paragraph("Yearly Precipitation Anomalies")
doc.add_picture("ppt_anomalies.png", width=Inches(6))

doc.add_paragraph("Yearly Tmax Anomalies")
doc.add_picture("tmax_anomalies.png", width=Inches(6))

doc.add_paragraph("Yearly Tmin Anomalies")
doc.add_picture("tmin_anomalies.png", width=Inches(6))

doc.add_paragraph("Seasonal PPT Anomalies")
doc.add_picture("seasonal_ppt_anomalies.png", width=Inches(6))

# Save Word File
doc.save("Climate_Anomalies_Report.docx")

print("✅ Processing complete: Excel + plots + Word report generated.")
